# Auxiliar 1 


----------------------------------

## 📚 Objetivos de la clase 📚

El objetivo principal de esta clase es introducirlos a la clasificación de texto en NLP. 
Para esto, implementaremos varios modelos de clasificación destinados a **predecir la categoría de noticias de la radio biobio**.

Los modelos y métodos que usaremos serán las vistas en las clases anteriores: 

- Preprocesamiento: Tokenización, Stemming, Lematización y eliminación de Stop Words.
- Bag of Words.
- Claisifcador de Bayes .
- Logistic regression.

La clase estará enfocada en utilizar las siguientes librerías (muy utilizadas en NLP):

- Pandas
- Scikit-Learn
- Spacy
- NLTK

Una vez resuelto, pueden utilizar cualquier parte del código que les parezca prudente para la tarea 1 (que también es de clasificación de texto! 😊).


El notebook del auxiliar ya ejecutado se encuentra en el [github](https://github.com/dccuchile/CC6205/tree/master/tutorials) del curso (Recuerden dejar su Star ⭐😉!).


## Importar las librerías

### En local: Python y Conda

Primero, si es que no tienen aun las librerías, hay que instalarlas.
Recuerden que usaremos `python 3.7` junto a `conda` como gestor de paquetes para el curso.

Este pueden descargarlo e instalenlo desde aquí : [🐍 Anaconda 🐍](https://www.anaconda.com/distribution/).

Para instalar las librerías, ejecutar en una consola 💻:

```cmd
conda install pandas scikit-learn ntkl spacy 
```
Y luego descargar el modelo de spacy en español: 

```cmd
python -m spacy download es_core_news_sm
```

Si saben un poco mas de anaconda, pueden instalar sus paquetes en un ambiente exlcusivo para el curso. Pero no es necesario!! Mas información [aquí](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html).


### Importar

In [1]:
import pandas as pd    
import spacy
import nltk

In [2]:
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

In [78]:
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

from spacy.lang.es.stop_words import STOP_WORDS
nlp = spacy.load("es_core_news_sm", disable=['ner', 'parser', 'tagger'])

## Clasificación de Texto 

¿Cuál de estos emails es SPAM?

![respuesta spam](https://raw.githubusercontent.com/dccuchile/CC6205/master/tutorials/recursos/spam.PNG "Email: Respuesta de cuales son spam y cuales no")


La respuesta es...

![respuesta spam](https://raw.githubusercontent.com/dccuchile/CC6205/master/tutorials/recursos/spam_2.PNG "Email: Respuesta de cuales son spam y cuales no")


Pero, estos también pueden representar otros tipos de categorías...

![categorías de comunicados](https://raw.githubusercontent.com/dccuchile/CC6205/master/tutorials/recursos/emails_clases.PNG "Email: Cual es la clase del email")

--------------------------
¿Cómo habrán encontrado la película?

![¿Cómo encontraron la película?](https://raw.githubusercontent.com/dccuchile/CC6205/master/tutorials/recursos/limpiapiscinas.PNG "Email: ¿Cómo encontraron la película?")


Créditos a [OndaMedia](https://ondamedia.cl/). Películas y materíal audiovisual chileno gratis, muy recomendado!

-------------------


Entonces, 

### ¿Qué es la clasificación de texto?

La clasificación de texto consiste en tomar distintos textos y asignarles alguna clase. Dichas clases varían según la task que queramos resolver. Por ejemplo:

    - Detectar emails SPAM -> SPAM, NO SPAM
    - Reviews de Peliculas -> Buena, Mas o menos, Mala, Malísima, Brutalmente mala.
    - Análisis de sentimientos de tweets: Felicidad, Tristeza, Enojo, Ira,...
    - Detectar Fake News -> Es, No es
    - Lenguaje del texto -> Español, Inglés, Chino,...
    - Categoría de una noticia -> Nacional, Internacional, Economía, Sociedad, Opinión...
    - Autor de un texto -> Cada autor es una clase distinta.
Se define formalmente como:

- Input: 

    - Un documento $d$
    - Un conjunto fijo de clases $c_1, c_2, ..., c_j$

- Output: 
    
    - Una clase $c \in C$ para el documento 
    
 
Hay dos clases de métodos para resolver estos problemas: 

1. **Hand-coded Rules 🤙**: 

    Establecemos a mano las reglas que permiten detectar las clases.
    

2. **Supervised Machine Learning 💻**:
   
    Entrenamos clasificadores a partir de muchos ejemplos de documentos etiquetados a mano. 
    

----------------------------------------


## ¿Qué haremos a continuación?


Cómo dijimos al comienzo, en este auxiliar crearemos un sistema que nos permita clasificar noticias de la radio biobio en 20 categorías o tópicos:

```python
[
    'america-latina', 'eeuu', 'europa', 'chile', 'region-metropolitana',
    'region-del-bio-bio', 'negocios-y-empresas', 'region-de-los-lagos',
    'actualidad-economica', 'region-de-valparaiso', 'region-de-la-araucania',
    'curiosidades', 'asia', 'region-de-los-rios', 'entrevistas', 'debates',
    'mediooriente', 'viral', 'animales', 'tu-bolsillo'
]
```

Los pasos a seguir serán: 

1. Primero que nada, descargaremos los datos con los que trabajaremos.

2. Luego, crearemos el sistema mas básico. Este consiste en transformar nuestro texto a `Bag of Words (BoW)` y luego, usar esos vectores para entrenar un clasificador. Este sistema nos puede entregar un muy buen baseline para comenzar a mejorar.

3. Evaluaremos nuestro clasificador según las métricas.

4. A continuación, veremos como mejorar aun mas nuestros resultados. Para esto agregaremos muchas mas técnicas vistas en cátedra, tales como el preprocesamiento de texto y probar con clasificadores aún mas sofisticados.



### Cargar los datasets 


Los datos que usaremos son 5000 documentos con noticias dividas en 20 categorías. Las noticias fueron obtenidas desde la página de la radio biobio.
Cada categoría contiene 250 documentos (noticias). 

Los cargaremos directamente desde el github del curso utilizando la librería `pandas` 🐼: 


In [57]:
dataset = pd.read_json('https://github.com/dccuchile/CC6205/releases/download/Data/biobio_clean.bz2')
dataset_r = dataset.copy(deep=True) # lo dejaremos ahí por si acaso.

In [58]:
# creamos una nueva columna titulo y contenido.
content = dataset['title'] + '. ' + dataset['content'] 
# obtenemos las clases
subcategory = dataset.subcategory

# dejamos en el dataset solo contenido de la noticia y categoria
dataset = pd.DataFrame({'content': content, 'category': subcategory})

In [59]:
# El número de noticias por clase lo pueden cambiar despues modificando la constante NUM_SAMPLES.
# noten que el número de noticias en el dataset original por categoría está desbalanceada.
# sample intentará sacar la mayor cantidad de ejemplos y retornará siempre, incluso si devuelve 
# menos de los que le pidieron.

NUM_SAMPLES = 250

categorias_seleccionadas = [
    'america-latina', 'eeuu', 'europa', 'chile', 'region-metropolitana',
    'region-del-bio-bio', 'negocios-y-empresas', 'region-de-los-lagos',
    'actualidad-economica', 'region-de-valparaiso', 'region-de-la-araucania',
    'curiosidades', 'asia', 'region-de-los-rios', 'entrevistas', 'debates',
    'mediooriente', 'viral', 'animales', 'tu-bolsillo'
]

# filtrar solo categorias seleccionadas
dataset = dataset[dataset['category'].isin(categorias_seleccionadas)]

# balancear clases
g = dataset.groupby('category')
dataset = pd.DataFrame(
    g.apply(lambda x: x.sample(NUM_SAMPLES).reset_index(drop=True))).reset_index(
        drop=True)

In [60]:
# así quedó nuestro dataset:
dataset.category.value_counts()

region-de-valparaiso      250
europa                    250
region-metropolitana      250
animales                  250
region-de-la-araucania    250
curiosidades              250
negocios-y-empresas       250
america-latina            250
region-de-los-lagos       250
tu-bolsillo               250
region-de-los-rios        250
eeuu                      250
mediooriente              250
chile                     250
actualidad-economica      250
entrevistas               250
region-del-bio-bio        250
asia                      250
viral                     250
Name: category, dtype: int64

Veamos unos cuantos ejemplos: 

In [61]:
dataset.sample(10)

,content,category
1859,Exdirector del Sename llama a mejorar Ley de a...,entrevistas
3586,Cuidador de niño asegura que no lo devolverá a...,region-de-valparaiso
2601,Casinos tributaron más de $10 mil millones en ...,negocios-y-empresas
2163,Barco con 233 migrantes rescatados atraca en M...,europa
2728,Liberan ranking mundial de aerolíneas 2018: un...,negocios-y-empresas
3325,Anciana abandonada en Valdivia: vecinas denunc...,region-de-los-rios
3353,Vecinos de Corral aseguran que la bahía estarí...,region-de-los-rios
512,7 tips que necesitas saber para que tus mascot...,animales
3815,Enfrentamientos en la UdeC dejan un carabinero...,region-del-bio-bio
4569,Audio 8D: descubre el sonido que hará volar tu...,viral


Ahora, el procedimiento estandar de introducción a la programación: **Dividir nuestros conjuntos en train y test.**

In [62]:
X_train, X_test, y_train, y_test = train_test_split(dataset.content,
                                                    dataset.category,
                                                    test_size=0.33,
                                                    random_state=42)

### Nuestro primer sistema de clasificación


Ahora que tenemos cargado el dataset, podemos implementar nuestro clasificador!

Para esto, usaremos 3 herramientas fundamentales de scikit-learn: un `pipeline`, `CountVectorizer` y `MultinomialNB`.

#### Pipeline 


Un [`pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) es la definición de los procesos que llevará a cabo el sistema que creemos. Nos permite tener unificados todos los procesos a la vez que simplifica el código de nuestro sistema.


En nuestro caso, el pipeline será:

    Dataset -> Bag of Words -> NaiveBayes Clf


#### Bag of Words y CountVectorizer 🎒 


¿Qué era Bag of Words?

Es un modelo en donde transformamos cada una de las oraciones de nuestro dataset en vectores. Cada vector contiene una columna por cada palabra / **token** del vocabulario. Al procesar el dataset, cada oración es mapeada a un vector que cuenta las apariciones de cada una de sus tokens. 

Referencia: [BoW en wikipedia](https://es.wikipedia.org/wiki/Modelo_bolsa_de_palabras)

**Un pequeño ejemplo**

Supongamos que nuestro tokenizador solo separa por espacios.

    - Doc1 : 'I love dogs'
    - Doc2: 'I hate dogs and knitting.
    - Doc3: 'Knitting is my hobby and my passion.

El bag of words quedaría:

<img src="https://i1.wp.com/datameetsmedia.com/wp-content/uploads/2017/05/bagofwords.004.jpeg" alt="BoW" style="width: 600px;"/>

`CountVectorizer` es la clase de `scikit` que transformará nuestro texto a Bag of Words. Fijense que es tremendamente útil tenerla dentro de un pipeline ya que fija en un comienzo el vocabulario que tendrá el Bag of Words, evitando discordancias entre los vectores del conjunto de entrenamiento y el de prueba.

#### MultinomialNB

Vamos a explicar el clásificador de Bayes enfocados en nuestro problema:

Escojamos un noticia, por ejemplo $d$ = `'7 tips que necesitas saber para que tus mascota...'`

Y nuestro conjunto de clases $C$=`{america-latina, eeuu, chile, ..., virales, animales}`

**Teorema de Bayes**

Podemos usar el teorema de bayes para calcular la probabilidad de que una noticia pertenezca a una de nuestras clases como:

$$p( c_i | d) = \frac{p(d | c_i) * p(c_i)}{p(d)}$$


Lo que puede ser escrito en nuestro ejemplo como: 


p( `america-latina` | `7 tips...`) = p( `7 tips...` | `america-latina`) * p(`america-latina`) /  p(`7 tips...`)

**Clasificación**


Por lo tanto, la clase de cada noticia quedará representada simplemente como la máxima probabilidad que obtengamos al calcular bayes para todas las clases $c_i$, lo que puede ser representado como un argmax:

$$argmax_{c_i \in C} p(c_i|d)$$

Ahora, despejemos un poco la ecuación:

1. Primero, veamos el denominador p(d) no nos entrega nada de información, ya que todas las noticias son distintas. Es decir $p(d) = \frac{1}{numero\ de\ docs}$

$$argmax_{c_i \in C} p(c_i|d) = argmax_{c_i \in C} p(d | c_i) * p(c_i)$$

2. Por otra parte, como nuestras clases están balanceadas, $p(c_i) = 250/5000 = 1 / 20$ . Nota que esto no siempre ocurre. Como puede pasar esto, dejaremos por ahora ese término dentro de nuestra ecuación.

Ahora ¿Cómo calculamos $p(d | c_i)$?:

1. Primero, separamos el documento por tokens. Menos mal que ya lo habíamos hecho conviertiendo nuestros documentos BoW...

$$p(d | c_i) = p(x_1, x_2, x_n | c_i)$$

2. Ahora, asumimos que las probabilidades de que cada una de las palabras pertenezca a la clase c_i es independiente de cualquier otra, es decir: 

$$ p(x_1, x_2, x_n | c_i) = p(x_1 | c_i) * p(x_2 | c_i) * ... * p(x_n | c_i)$$

Observen que ahora, dado que asumimos esto, el orden de las palabras no importa. Otro punto mas a favor de usar BoW.

**Entrenamiento 🥋**

Y aquí viene la parte del entrenamiento del modelo: para calcular $p(x | c_i)$ simplemente contamos la cantidad de veces que aparece esa palabra en los documentos de entrenamiento de la clase c_i. O sea:

$$p(x | c_i) = \frac{count(x, c_i)}{\sum_{x_i \in Vocab} count(x_i, c_i)}$$

Qué pasa si hay una palabra que no vimos en el entrenamiento de la clase c_i?

$p(x | c_i) = 0$ y se anula todo!

Podemos aplicar suavizamientos para evitar este tipo de problemas

$$p(x | c_i) = \frac{count(x, c_i) + 1 }{\sum_{x_j \in Vocab} count(x_j, c_i) + |Vocab|}$$



Juntando lo anterior, nuestro problema ahora se transforma a :

$$c = argmax_{c_i \in C} \prod_{x_j \in d} P(x_j | c_i) * p(c_i)$$

Sin embargo, estas probabilidades son muy pero muy pequeñas cuando tenemos vectores Bag of Words muy grandes. Imaginense un vocabulario de 100.000 palabras. Además, multiplicar es muy complicado ❌😥

Así que convertimos nuestra multiplicatoria en una suma de logaritmos!

$$c = argmax_{c_i \in C}\  log(p(c_i)) + \sum_{x_i \in d} log(P(x_i | c_i))$$



Y con eso, ya tenemos definido como va a funcionar nuestro clasificador. Si bien, existen muchos mas clasificadores, este ya es lo suficientemente potente como para lograr muy buenos resultados.

------------------------
#### Creemos el clasificador 🧪

**Primero, definimos el pipeline**

In [63]:
# Definimos el vectorizador para convertir el texto a BoW:
vectorizer = CountVectorizer()  

# Definimos el clasificador que usaremos.
clf = MultinomialNB()   

# Creamos el pipeline
text_clf = Pipeline([('vect', vectorizer), ('clf', clf)])

**Luego, lo entrenamos**

In [64]:
# Entrenamos nuestro pipeline
text_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

**Y predecimos**

In [68]:
y_pred = text_clf.predict(X_test)

**Veamos como nos fue:**


In [69]:
# algunos ejemplos:
pd.DataFrame({'content': X_test, 'category':y_test, 'predicted category': y_pred}).sample(10)

,content,category,predicted category
3467,Vecinos de Melefquén exigen claridad a las aut...,region-de-los-rios,region-de-los-rios
1200,Idea de legislar reforma a las pensiones se vo...,chile,chile
642,Dueño busca a los responsables del asesinato d...,animales,animales
3938,Desconocidos rompen vidrios y lanzan pintura c...,region-del-bio-bio,region-del-bio-bio
4561,Mario: el enternecedor niño maestro de las ven...,viral,viral
3604,Camioneros anuncian querella criminal tras que...,region-de-valparaiso,region-de-valparaiso
2377,Irán acusó a Israel de inventar pretextos para...,mediooriente,mediooriente
3576,Pronostican bajas temperaturas en Valparaíso t...,region-de-valparaiso,chile
3570,Menor de 12 años queda con fractura expuesta t...,region-de-valparaiso,region-de-valparaiso
1694,Migrantes indocumentados esperan angustiados l...,eeuu,eeuu


In [148]:
# usando la matriz de confusión:

# eje x -> predichos
# eje y -> clase real

print(confusion_matrix(y_test, y_pred))

[[61  0  0  0  3  2  0  1  1  0 11  0  1  1  0  1  0  8  1]
 [ 1 71  1  0  3  2  5  0  3  0  0  0  0  0  0  0  0  0  3]
 [ 0  0 63  1  0  5  0  0  0  0  0  0  1  0  0  0  0  0  5]
 [ 0  2  1 64  0  3  1  0  1  4  0  0  0  0  0  0  0  1  1]
 [10  2  0  1 55  2  1  1  1  0  4  2  2  2  4  4  6  3  1]
 [ 2  1  1  4  0 37  2  0  1  1  2  0  1  1  1  0  0  2 24]
 [ 1  7  0  9  0  4 56  0  2  2  3  0  0  0  0  0  0  0  2]
 [ 0  0  0  0  0  0  0 74  0  0  1  0  0  0  0  0  0  0  0]
 [ 1  4  4  1  0  3  3  0 65  3  1  0  0  0  0  0  1  0  2]
 [ 0  1  1  1  0  0  9  0  1 71  0  0  0  0  0  0  0  0  1]
 [13  1  0  0  2  3  0  0  0  0 45  0  0  0  1  3  1  7  2]
 [ 0  0  0  0  2  1  0  0  0  0  0 72  3  0  3  1  2  2  0]
 [ 0  1  0  0  1  0  0  0  0  0  0  0 70  1  2  2  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  3 74  1  0  1  0  0]
 [ 1  0  1  0  3  0  0  1  0  0  2  0  0  2 63  1  1  0  1]
 [ 0  0  0  0  1  1  0  1  0  0  1  2  2  0  1 69  1  0  0]
 [ 1  1  0  0  4  0  0  0  0  0  3  0  3

#### Métricas de Evaluación

Las métricas definen un puntaje de evaluación que indica que tal le fue al sistema. Hay muchas formas distintas de medir su rendimiento. Entre estas, tenemos:

- `precision`: El número de documentos de una clase clasificados correctamente dividido por el número de documentos totales clasificados como esa clase.

- `recall`: El número de documentos de una clase clasificados correctamente dividido por el número de los documentos que se deberían haber clasificado como esa clase.(número de documentos reales de esa clase).

- `f1-score` : Es la media armónica entre los anteriores.

- `accuracy` : La cantidad de documentos clasificados correctamente versus todos los documentos

Por otra parte, tenemos dos formas de ver dichas métricas agrupadas:

- `Macroaveraging`:    Se computan las métricas por cada clase y luego de promedia.

- `Microaveraging`:    Se recolectan las clasificaciones por cada clase, se computa la tabla de contingencia (todos los elementos clasificados) y se evalua. Representa un Macroaveraging ponderado por el número de miembros de una clase.

In [72]:
# usando el classification report:
print(classification_report(y_test, y_pred))

                        precision    recall  f1-score   support

  actualidad-economica       0.56      0.70      0.62        91
        america-latina       0.82      0.84      0.83        89
              animales       0.75      0.91      0.82        75
                  asia       0.75      0.82      0.79        78
                 chile       0.48      0.58      0.53       101
          curiosidades       0.49      0.62      0.55        80
                  eeuu       0.74      0.73      0.74        86
           entrevistas       0.79      0.93      0.85        75
                europa       0.92      0.69      0.79        88
          mediooriente       0.80      0.93      0.86        85
   negocios-y-empresas       0.57      0.60      0.59        78
region-de-la-araucania       1.00      0.42      0.59        86
   region-de-los-lagos       0.81      0.70      0.75        77
    region-de-los-rios       0.84      0.72      0.78        80
  region-de-valparaiso       0.66      

#### Ejecutemos algunas consultas!

In [87]:
text_clf.predict([
    ("En puerto montt se encontró un perrito, que aparentemente,"
    "habría consumido drogas de alto calibre. Producto de esto,"
    "se ponostíca que padecerá severa caña durante varios dias.")
])

array(['region-de-los-lagos'], dtype='<U22')

In [88]:
text_clf.predict(["kim jong un será el próximo candidato a ministro de educación."])

array(['asia'], dtype='<U22')

In [89]:
text_clf.predict([("El banco mundial presentó para chile un decrecimiento"
                   "económico de 92% y una inflación de 8239832983289%.")])

array(['actualidad-economica'], dtype='<U22')

-------------------------

Se ven bastante buenos los resultados. ¿Pero, podremos mejorarlos?

### Preprocesamiento del texto

En clases vimos que habían varias técnicas que permiter preprocesar los textos.
Es decir, cómo hacemos el proceso de tokenización (separación de las palabras).



Alguna de las técnicas son:


- Eliminación de Stopwords
- Stemming
- Lematización

Existen otros preprocesadores que agregan información a las oraciones, tales como aquellos que indican negaciones.

A continuación, describiremos con mas detalle cada uno de estas técnicas.

#### Tokenizar ➗

¿Qué era tokenizar?


    Es el proceso de convertir una secuencia de carácteres (por ejemplo, una oración) en una secuencia de valores distintos entre si llamados tokens.
    
Referencia: [Tokenización en wikipedia](https://en.wikipedia.org/wiki/Lexical_analysis#Tokenization)



**spaCy y el objeto nlp**

`nlp` es el objeto que nos permite usar e interactuar con la librería [`spacy`](https://spacy.io/).
Esta librería incluye variadas herramientras, tales como tokenizar, lematizar, descartar stopwords, entre otras (para este auxiliar, solo utilizaremos las mencionadas). El objeto nlp lo instanciamos en la sección de imports.

Para usarla, simplemente se le pasa el texto como parámetro, como veremos en el siguiente ejemplo: 

In [79]:
DOC = "hermanito mio te estas pegando el show"

tokens = []
for word in nlp(DOC):
    tokens.append(word)

tokens

[hermanito, mio, te, estas, pegando, el, show]

**Observación**: Para este auxiliar usaremos `List Comprehension`, otra forma de hacer un for un poco mas reducida.
Una muy buena referencia de esto [aquí](https://www.programiz.com/python-programming/list-comprehension).

La operación anterior usando esta sintáxis quedaría como:

In [80]:
tokens = [word for word in nlp(DOC)]
tokens

[hermanito, mio, te, estas, pegando, el, show]

#### Stopwords 🛑

¿Qué eran las stopwords?

    Las Stopwords son palabras muy comunes en nuestro lenguaje y que por lo tanto, no aportan mucha información. Existen múltiples listas de stopwords para muchos idiomas y la aplicación de estas variará caso a caso.

    
Referencias: [Stopwords en Wikipedia](https://en.wikipedia.org/wiki/Stop_words)

En este caso, utilizaremos las stopwords inlcuidas en la librería spaCy en español

In [81]:
print(len(STOP_WORDS))
print(list(STOP_WORDS)[0:20])

551
['sido', 'quienes', 'parte', 'conseguimos', 'conocer', 'estará', 'haya', 'mías', 'total', 'breve', 'donde', 'nosotras', 'nuestros', 'ni', 'tus', 'realizar', 'lugar', 'esos', 'última', 'aquella']


#### Stemming 🔪

¿Qué era el stemming? 

    Son un conjunto de métodos enfocados en reducir cada palabra a su raiz.

Referencia: [Stemming en Wikipedia](https://en.wikipedia.org/wiki/Stemming)
  
**Ejemplos: **


| word | stem of the word  |
|---|---|
working | work
worked | work
works | work

**nltk**

En este caso, utilizaremos la segunda librería de herramientas de nlp: [`nltk`](https://www.nltk.org/). Esta provee una buena herramienta para hacer stemming en español : `SnowballStemmer`

In [82]:
stemmer = SnowballStemmer('spanish')
stemmed_doc = [stemmer.stem(str(token)) for token in tokens]
print(stemmed_doc)

['hermanit', 'mio', 'te', 'estas', 'peg', 'el', 'show']


#### Lematización 🙀

¿Qué era lematización? 

    Es el proceso de transformar cada token a su lema, el cual es la palabra base sin ningún tipo de flexión o alteración como las conjugaciones, por ejemplo.
    
    

  
    
    
Referencia: [Lematización en wikipedia](https://en.wikipedia.org/wiki/Lemmatisation)

Refernecia: [Flexión en las palabras](https://es.wikipedia.org/wiki/Flexi%C3%B3n_(ling%C3%BC%C3%ADstica))

**Ejemplos**

| word | lemma  |
|---|---|
dije| decir 
guapas | guapo
mesa | mesas


 <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/29/Flexi%C3%B3nGato-svg.svg/300px-Flexi%C3%B3nGato-svg.svg.png" alt="Flexión de gato" style="width: 200px;"/>
 

**Lematizar el texto**

Al igual que la tokenización, utilizaremos `scpaCy` (a través del objeto `nlp`) para lematizar el contenido.

In [83]:
lemmatized_content = [word.lemma_ for word in nlp(DOC)]
print(lemmatized_content)

['hermanito', 'mio', 'te', 'este', 'pegar', 'el', 'show']



**Discusión:**

    ¿Cuál es mejor?

### Sistema de clasificación con preprocesamiento

Para agregar los tokenizadores en el sistema, creamos funciones que que cada documento de forma individual usando nuestro preprocesador favorito. Luego, `CountVectorizer` se encargará de usar estas funciones sobre todo el dataset.

#### Tokenizadores para CountVectorizer

In [27]:
# Tokenizers para CountVectorizer

# Solo tokenizar el doc usando spacy.
def tokenizer(doc):
    return [x.orth_ for x in nlp(doc)]


# Tokenizar y remover las stopwords del doc
def tokenizer_with_stopwords(doc):
    return [x.orth_ for x in nlp(doc) if x.orth_ not in STOP_WORDS]


# Tokenizar y lematizar.
def tokenizer_with_lemmatization(doc):
    return [x.lemma_ for x in nlp(doc)]

# Tokenizar y hacer stemming.
def tokenizer_with_stemming(doc):
    stemmer = SnowballStemmer('spanish')
    return [stemmer.stem(word) for word in [x.orth_ for x in nlp(doc)]]

#### Creamos nuestro clasificador


**Definimos el pipeline**




In [84]:
# seleccionamos el tokenizador a usar:
TOKENIZER = tokenizer_with_stemming

# Definimos el vectorizador para convertir el texto a BoW:
vectorizer = CountVectorizer(analyzer='word',
                             tokenizer=TOKENIZER,
                             ngram_range=(1, 1))

# Definimos el clasificador que usaremos.
clf = MultinomialNB()   

# Creamos el pipeline
text_clf_2 = Pipeline([('vect', vectorizer), ('clf', clf)])

**Entrenamos nuestro pipeline y predecimos**


In [85]:
text_clf_2.fit(X_train, y_train)
y_pred = text_clf_2.predict(X_test)

**Evaluamos**

In [86]:
# usando la matriz de confusión:
print(confusion_matrix(y_test, y_pred),
      '\n\n-------------------------------------------------------\n')
# usando el classification report:
print(classification_report(y_test, y_pred))

[[64  1  0  0  5  1  1  1  0  0 10  0  0  0  0  0  0  8  0]
 [ 0 76  1  3  0  5  2  1  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 67  1  0  3  0  0  0  0  0  0  0  0  0  0  0  0  4]
 [ 0  1  1 64  0  4  2  0  0  6  0  0  0  0  0  0  0  0  0]
 [10  5  0  1 54  1  1  6  2  0  7  0  3  2  2  3  2  2  0]
 [ 0  2  3  2  0 55  2  0  2  0  2  0  0  0  0  0  1  2  9]
 [ 0  4  0  9  0  6 59  0  1  6  0  0  0  0  0  0  0  0  1]
 [ 0  0  0  0  0  2  0 71  0  0  0  0  0  0  0  0  0  2  0]
 [ 1  1  4  6  1  5  3  0 62  5  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  2  3  0  0 79  0  0  0  0  0  0  0  0  0]
 [15  3  0  0  3  2  1  0  0  0 47  0  0  0  0  1  0  6  0]
 [ 1  0  0  0 13  0  0  2  0  0  1 37  1  9  6  0 13  3  0]
 [ 1  0  0  0  7  1  0  5  0  0  1  1 54  2  3  0  2  0  0]
 [ 1  0  0  0  8  0  0  1  0  0  3  1  7 52  3  0  3  1  0]
 [ 3  0  1  0  7  1  0  4  0  0  4  0  0  0 51  1  4  0  0]
 [ 6  0  0  0 10  2  0  3  0  0  3  1  2  1  8 34  8  1  0]
 [ 2  1  0  0 13  2  0  3  0  0  4  0  0

#### Pregunta abierta: ¿Por qué no mejoran los resultados?

[Aquí](https://www.quora.com/Is-it-normal-to-get-better-accuracy-without-stemming-and-lemmatization-than-using-them-in-NLP-text-classification) hay una muy buena discusión al respecto.

### Clasificación usando Regresión Logísitica

No profundizaremos en este clasificador, mas del hecho de que se "supone" que debería tener mejor rendimiento que el de bayes.

Referencia: [Regresión Logística](https://en.wikipedia.org/wiki/Logistic_regression)

**Definimos nuestro Pipeline**

In [146]:
# Qué tokenizer usaremos?
TOKENIZER = tokenizer_with_lemmatization

# Definimos el vectorizador para convertir el texto a BoW:
vectorizer = CountVectorizer(analyzer='word',
                             tokenizer=TOKENIZER,
                             ngram_range=(1, 1))

# Ahora definimos regresión logística como clasificador.
log_mod = LogisticRegression(solver='lbfgs', multi_class='ovr', max_iter = 1000)   
log_pipe = Pipeline([('vect', vectorizer), ('clf', log_mod)])

**Entrenamos y predecimos**

In [147]:
log_pipe.fit(X_train, y_train)
y_pred = log_pipe.predict(X_test)

**Evaluamos**

In [ ]:
# usando la matriz de confusión:
print(confusion_matrix(y_test, y_pred),
      '\n\n-------------------------------------------------------\n')
# usando el classification report:
print(classification_report(y_test, y_pred))

### N-gramas

Los n-gramas son conjuntos de n-tokens seguidos entre si. La idea de usar esto es que además, capturemos conceptos. 

Por ejemplo, si usamos 2-gramas sobre `'Hoy día comí lentejas'`, esta quedaría como:


```python
['hoy dia', 'día comí', 'comí lentejas']
```

`CountVectorizer` tiene la opción para poner n-gramas del tamaño que tu quieras, y además incluir mas pequeños. Todo esto se define en el parámetro `ngram_range`. Este recibe una tupla con los rangos del n-grama mas pequeño y el mas grande. Por ejemplo, para (1,2), la oración anterior quedaría: 


```python
['hoy', 'día', 'comí', 'lentejas', 'hoy dia', 'día comí', 'comí lentejas']
```

Nota que esto incrementa el tamaño de los vectores de Bag of words y por lo tanto, del entrenamiento y de la predicción. 

In [93]:
# Qué tokenizer usaremos?
TOKENIZER = tokenizer_with_lemmatization

# Definimos el vectorizador para convertir el texto a BoW:
vectorizer = CountVectorizer(analyzer='word',
                     
                             tokenizer=TOKENIZER,
                             ngram_range=(1, 3))

# Ahora definimos regresión logística como clasificador.
log_mod = LogisticRegression(solver='lbfgs', multi_class='ovr', max_iter = 1000)   
log_pipe = Pipeline([('vect', vectorizer), ('clf', log_mod)])

**Entrenamos y predecimos**

In [94]:
log_pipe.fit(X_train, y_train)
y_pred = log_pipe.predict(X_test)

**Evaluamos**

In [95]:
# usando la matriz de confusión:
print(confusion_matrix(y_test, y_pred),
      '\n\n-------------------------------------------------------\n')
# usando el classification report:
print(classification_report(y_test, y_pred))

[[53  0  0  0  6  2  0  1  0  0 15  0  1  0  0  2  0 10  1]
 [ 0 74  1  0  3  1  5  0  2  0  1  0  0  0  0  0  0  0  2]
 [ 0  0 66  1  0  2  0  0  0  0  0  0  0  0  0  0  0  0  6]
 [ 0  1  2 66  0  1  0  0  4  1  0  0  0  0  0  0  0  1  2]
 [ 9  1  0  1 58  1  0  1  1  0  4  1  2  2  4  5  9  1  1]
 [ 1  2  3  3  0 39  1  0  1  1  4  0  0  0  0  0  0  2 23]
 [ 0  6  0 10  0  4 57  0  1  2  2  1  0  0  0  0  0  0  3]
 [ 0  0  0  0  0  0  0 74  0  0  1  0  0  0  0  0  0  0  0]
 [ 1  3  4  4  0  3  3  0 64  2  1  0  0  0  0  0  1  0  2]
 [ 0  1  0  2  0  1  5  0  0 75  0  0  0  0  0  0  0  0  1]
 [13  1  0  0  1  3  0  0  0  0 53  0  0  0  1  1  0  2  3]
 [ 0  0  0  0  2  1  0  0  0  0  1 74  1  0  3  1  2  1  0]
 [ 0  1  0  0  2  0  0  1  0  0  0  0 68  0  2  2  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  2 76  0  0  1  0  0]
 [ 1  0  1  0  1  0  0  1  0  0  1  0  2  0 67  0  1  0  1]
 [ 0  0  0  0  0  2  0  1  0  0  2  1  2  0  2 67  2  0  0]
 [ 1  1  0  0  3  0  0  0  0  0  3  0  4

### Bonus: Clasificación de Autoría de documentos

¿Existirá un patrón en como escriben los periodistas que nos permitan identificarlos a partir de sus textos?

In [109]:
dataset_r.author.value_counts()[0:20]

Diego Vera               4471
Emilio Lara              1380
Matías Vega              1230
César Vega Martínez      1202
María José Villarroel    1157
Gonzalo Cifuentes        1122
Manuel Stuardo           1022
Manuel Cabrera            999
Valentina González        966
Nicole Briones            850
Felipe Delgado            825
Yessenia Márquez          764
Paola Alemán              760
Verónica Reyes            718
Jonathan Flores           618
Nicolás Díaz              592
Sebastián Asencio         544
Catalina Díaz             523
Ariela Muñoz              515
Nicolás Parra             503
Name: author, dtype: int64

In [125]:
NUM_SAMPLES = 250

def process_datasets_by_author(dataset):
    
    # creamos una nueva columna titulo y contenido.
    content = dataset['title'] + '. ' + dataset['content'] 
    # obtenemos las clases
    subcategory = dataset.author
    # dejamos en el dataset solo contenido de la noticia y categoria
    dataset = pd.DataFrame({'content': content, 'author': subcategory})

    selected_authors = ['Diego Vera', 'Emilio Lara', 'Matías Vega', 'César Vega Martínez',
           'María José Villarroel', 'Gonzalo Cifuentes', 'Manuel Stuardo',
           'Manuel Cabrera', 'Valentina González', 'Nicole Briones',
           'Felipe Delgado', 'Yessenia Márquez', 'Paola Alemán', 'Verónica Reyes',
           'Jonathan Flores', 'Nicolás Díaz', 'Sebastián Asencio', 'Catalina Díaz',
           'Ariela Muñoz', 'Nicolás Parra']

    # filtrar solo categorias seleccionadas
    dataset = dataset[dataset['author'].isin(selected_authors)]

    # balancear clases
    g = dataset.groupby('author')
    dataset = pd.DataFrame(
        g.apply(lambda x: x.sample(NUM_SAMPLES).reset_index(drop=True))).reset_index(
            drop=True)
    
    return dataset

    



In [127]:
author_dataset

,content,author
0,Gobierno evalúa aumentar hasta un 40% la cuota...,Ariela Muñoz
1,"Aeropuerto de Santiago mejoró en un 6,4% la pu...",Ariela Muñoz
2,"España pide ""sanciones"" de la Unión Europea co...",Ariela Muñoz
3,Gobierno firma convenio para tratar salud ment...,Ariela Muñoz
4,Vivienda se incendia por completo en La Arauca...,Ariela Muñoz
...,...,...
4995,"Armas de fogueo, desorden y amenazas generan a...",Yessenia Márquez
4996,Madre de joven que lleva 6 meses desaparecida ...,Yessenia Márquez
4997,Este lunes formalizarían a hombre acusado de p...,Yessenia Márquez
4998,Gobernador de Palena acusa atrasos en barcazas...,Yessenia Márquez


In [128]:
author_dataset = process_datasets_by_author(dataset_r.copy(deep=True))

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(
    author_dataset.content,
    author_dataset.author,
    test_size=0.33,
    random_state=42)

#### Definir el Pipeline

In [143]:
# Definimos el vectorizador para convertir el texto a BoW:
vectorizer = CountVectorizer(analyzer='word',
                             ngram_range=(1, 1))

# Definimos el clasificador. Usaremos bayes, ya que regresión logística se demora 1/4 del tiempo del universo.
clf = MultinomialNB()   


# Creamos el pipeline
log_pipe_by_author = Pipeline([('vect', vectorizer), ('clf', clf)])

#### Entrenar

In [144]:
log_pipe_by_author.fit(X_train_2, y_train_2)
y_pred = log_pipe_by_author.predict(X_test_2)

#### Evaluar

In [145]:
# usando la matriz de confusión:
print(confusion_matrix(y_test_2, y_pred),
      '\n\n-------------------------------------------------------\n')
# usando el classification report:
print(classification_report(y_test_2, y_pred))

[[ 5  0  1  8  0  6  1 26  0  0  6  3  1  3  3  9  3  3 15  1]
 [ 0 25  1  1  1  7  0  1  1  6  7  1  5 12  1  4  0  0  0 12]
 [ 0  0 55  8  0  0  0  5  0  0  0  0  0  0  0 22  0  0  0  0]
 [ 0  0  0 53  0  0  0  6  0  0  0  2  0  0  0 10  0  0  1  0]
 [ 1  2  2 34  2  1  0 27  2  2  3  0  0  4  1  4  0  0  9  2]
 [ 0  0  1 18  0 34  0  4  0  0  9  0  0  2  0  5  1  3  0  0]
 [ 0  0  2 28  1  4  1 28  1  0  4  2  0  5  0 10  0  0  5  1]
 [ 0  0  0 13  0  2  1 49  2  0  0  4  0  1  0  2  0  0  3  0]
 [ 0  1  0 10  0  4  0 12 17  0  2  1  1 18  5  5  1  0  1  4]
 [ 0  9  1  5  0  6  0  9  0  4 14  4  7 17  1  0  0  0  1  7]
 [ 1  0  1  5  0  4  0 13  1  0 24  1  1 12  2  4  1  4  2  1]
 [ 0  1  1 19  0  2  1 26  0  2  1  5  0  5  4  8  3  0  5  0]
 [ 0  8  0  5  0  3  0  6  1  4  8  1 11 15  1  2  0  0  0 22]
 [ 0  2  0  1  1  9  1 13  1  0  5  0  0 36  3  0  0  0  0  1]
 [ 1  3  1 18  0  3  0 11  7  0  4  1  0 11 16  1  0  0  3  0]
 [ 0  0  1 21  0  1  0  4  0  0  0  0  0  0  0 53  1  0

## Créditos

Todas las noticias extraidas perteneces a [Biobio Chile](https://www.biobiochile.cl/), los cuales gentilmente licencian todo su material a través de la [licencia Creative Commons (CC-BY-NC)](https://creativecommons.org/licenses/by-nc/2.0/cl/)

## Referencias

Gitgub del curso: 
- https://github.com/dccuchile/CC6205

Slides:
- https://web.stanford.edu/~jurafsky/slp3/slides/7_NB.pdf


Análisis de sentimientos como clasificación de texto:
- https://affectivetweets.cms.waikato.ac.nz/benchmark/

Algunos Recursos útiles
- [Pandas Cheat Sheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)
- [Scikit-learn Cheat Sheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Scikit_Learn_Cheat_Sheet_Python.pdf)
- [Spacy Tutorial](https://www.datacamp.com/community/blog/spacy-cheatsheet)
- [NLTK Cheat sheet](http://sapir.psych.wisc.edu/programming_for_psychologists/cheat_sheets/Text-Analysis-with-NLTK-Cheatsheet.pdf)